In [9]:
from pyspark.sql import SparkSession
from vars import *
from datetime import date
from functions import loadConfigs
import os
from pyspark.sql.functions import explode
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from delta import *

# minio credentials
minio_endpoint = "http://minio:9000"
minio_access_key = "iQekrU3nA33aKktu2GMo"
minio_secret_key = "1OGQ3m1lWEZVQ31xKfPCKmZVFVb2lXwCorjZr1d1"
minio_region = "us-east-1"
minio_bucket = "reddit-datalake"

builder = loadConfigs(SparkSession.builder)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [10]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [20]:
# today = date.today().strftime('%Y%m%d')
today = 20230610

input_folder = "gildings"
input_file = "gildings"
mode = "append"

df = spark.read.option("header", "true") \
    .parquet(f"s3a://{minio_bucket}/silver/{input_folder}/{input_file}_{today}")

In [21]:
df.show()

+-------+-------------------+------+------------------+-----------+---------+-------------+--------+-----+------------+--------------------+--------+
|post_id|             author|gilded|          gildings|gild_silver|gild_gold|gild_platinum|is_video|score|subreddit_id|           subreddit|  dateid|
+-------+-------------------+------+------------------+-----------+---------+-------------+--------+-----+------------+--------------------+--------+
|145hgit|TheUndeadGunslinger|     1|      {1, 1, null}|          1|        1|         null|   false|20451|    t5_2qh03|              gaming|20230610|
|145hijg|     yycsoftwaredev|     0|{null, null, null}|       null|     null|         null|   false| 3123|    t5_2qh68|              canada|20230610|
|145uv3p|            esberat|     0|{null, null, null}|       null|     null|         null|    true| 5147|    t5_qir9n|               HolUp|20230610|
|145owhy|        BigHoneyBot|     0|{null, null, null}|       null|     null|         null|   false|

In [22]:
author_df = df.groupBy("author","subreddit_id","subreddit").agg(
    F.sum("gild_silver").alias("total_gild_silver"),
    F.sum("gild_gold").alias("total_gild_gold"),
    F.sum("gild_platinum").alias("total_gild_platinum"),
    F.sum("score").alias("total_score")
)

In [23]:
author_df.show()

+--------------------+------------+-----------------+-----------------+---------------+-------------------+-----------+
|              author|subreddit_id|        subreddit|total_gild_silver|total_gild_gold|total_gild_platinum|total_score|
+--------------------+------------+-----------------+-----------------+---------------+-------------------+-----------+
|           uchman365|    t5_2r5rp|         facepalm|             null|           null|               null|       9702|
|  Aware-Vacation6570|   t5_5rj68d|   popculturechat|             null|           null|               null|       4811|
|     FionaWalliceFan|    t5_3md1s|     MovieDetails|             null|           null|               null|       7283|
|  povitryana_tryvoga|    t5_2v0c6|    CombatFootage|             null|           null|               null|       3683|
|            TitusRex|    t5_2to41|             tifu|                1|             18|                  8|      67842|
|        cooladriyana|   t5_5tdqj0|     

In [24]:
author_df = author_df.withColumn(
    'total_gildings',
    F.col('total_gild_silver') + F.col('total_gild_gold') + F.col('total_gild_platinum')
).orderBy('total_gildings',ascending=False)

In [25]:
author_df.show()

+------------------+------------+-----------------+-----------------+---------------+-------------------+-----------+--------------+
|            author|subreddit_id|        subreddit|total_gild_silver|total_gild_gold|total_gild_platinum|total_score|total_gildings|
+------------------+------------+-----------------+-----------------+---------------+-------------------+-----------+--------------+
|          TitusRex|    t5_2to41|             tifu|                1|             18|                  8|      67842|            27|
|        OBLIVIATER|    t5_2qh1e|           videos|                1|              5|                  2|      14796|             8|
|            _alco_|    t5_2qh3v|           bestof|                2|              5|                  1|      61604|             8|
|       sussywanker|    t5_2qh16|       technology|                1|              2|                  2|      69008|             5|
|         uchman365|    t5_2r5rp|         facepalm|             null|

In [28]:
from pyspark.sql.functions import current_date, date_format

author_df = author_df.withColumn("updated_at", date_format(current_date(), "yyyyMMdd"))

In [29]:
author_df.printSchema()

root
 |-- author: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- total_gild_silver: long (nullable = true)
 |-- total_gild_gold: long (nullable = true)
 |-- total_gild_platinum: long (nullable = true)
 |-- total_score: long (nullable = true)
 |-- total_gildings: long (nullable = true)
 |-- updated_at: string (nullable = false)



In [17]:
post_df = df.groupBy("post_id").agg(
    F.sum("gild_silver").alias("total_gild_silver"),
    F.sum("gild_gold").alias("total_gild_gold"),
    F.sum("gild_platinum").alias("total_gild_platinum")
)

In [11]:
post_df = post_df.withColumn(
    'total_gildings',
    F.col('total_gild_silver') + F.col('total_gild_gold') + F.col('total_gild_platinum')
)

In [12]:
post_df.orderBy('total_gildings',ascending=False).show()

+-------+-----------------+---------------+-------------------+--------------+
|post_id|total_gild_silver|total_gild_gold|total_gild_platinum|total_gildings|
+-------+-----------------+---------------+-------------------+--------------+
|145efj4|                1|             18|                  8|            27|
|145ea4f|                2|              5|                  1|             8|
|145vns0|                1|              5|                  2|             8|
|145hyey|                1|              2|                  2|             5|
|145ji4h|             null|           null|               null|          null|
|145t1sa|             null|           null|               null|          null|
|145nhgv|             null|           null|               null|          null|
|145opim|             null|           null|               null|          null|
|145p0w0|             null|           null|               null|          null|
|145bv3e|             null|           null|         

In [13]:
from pyspark.sql.functions import current_date, date_format

author_df = author_df.withColumn("updated_at", date_format(current_date(), "yyyyMMdd"))

In [14]:
author_df.show()

+--------------------+-----------------+---------------+-------------------+--------------+----------+
|              author|total_gild_silver|total_gild_gold|total_gild_platinum|total_gildings|updated_at|
+--------------------+-----------------+---------------+-------------------+--------------+----------+
|            TitusRex|                1|             18|                  8|            27|  20230611|
|              _alco_|                2|              5|                  1|             8|  20230611|
|          OBLIVIATER|                1|              5|                  2|             8|  20230611|
|         sussywanker|                1|              2|                  2|             5|  20230611|
|     PoopyMcAllister|             null|           null|               null|          null|  20230611|
|      TrixoftheTrade|             null|           null|               null|          null|  20230611|
|        cooladriyana|             null|           null|               nu

In [15]:
author_df.columns

['author',
 'total_gild_silver',
 'total_gild_gold',
 'total_gild_platinum',
 'total_gildings',
 'updated_at']

In [11]:
output_folder = "author_gildings"
output_file = "author_gildings"

In [13]:
author_df.write.format("delta").mode("overwrite").save(f"s3a://{minio_bucket}/gold/{output_folder}/{output_file}_{today}")

In [19]:
df = spark.read.option("header", "true") \
    .parquet(f"s3a://{minio_bucket}/silver/popular/popular_{today}")

In [20]:
df.show()

+---------------+--------+--------------------+---------------+--------------+--------+--------+-------+-------------+-------------+---------------+-------------+------------------+-----+------------+------+------+----------+-------+----------------------+----------------+----------+-------+-------------------+------------------+-------+--------+-----+------+----------+---------+------------+--------------+-----------+-------+-----------------------+--------------------+------+------------+----+----------+--------------+-----+-----+--------------------+--------------------+------------+-------+--------+--------------------+------------+-----------------------+---------------------+--------------+--------------+--------------------+--------------------+----------------+---------------------+-----+------------+----------+-------+----------------+----+--------+
|approved_at_utc|archived|              author|author_fullname|author_premium|can_gild|category|clicked|      created|  created_u

In [14]:
delta_df = spark.read.format("delta").load(f"s3a://{minio_bucket}/gold/{output_folder}/{output_file}_{today}")

In [15]:
delta_df.show()

+------------------+------------+-----------------+-----------------+---------------+-------------------+--------------+
|            author|subreddit_id|        subreddit|total_gild_silver|total_gild_gold|total_gild_platinum|total_gildings|
+------------------+------------+-----------------+-----------------+---------------+-------------------+--------------+
|          TitusRex|    t5_2to41|             tifu|                1|             18|                  8|            27|
|        OBLIVIATER|    t5_2qh1e|           videos|                1|              5|                  2|             8|
|            _alco_|    t5_2qh3v|           bestof|                2|              5|                  1|             8|
|       sussywanker|    t5_2qh16|       technology|                1|              2|                  2|             5|
|         uchman365|    t5_2r5rp|         facepalm|             null|           null|               null|          null|
|Aware-Vacation6570|   t5_5rj68d